In [7]:
import os
import requests as rq
import pandas as pd
import numpy as np
import json
import re
from bs4 import BeautifulSoup
from tqdm import tqdm,trange
from PIL import Image

In [2]:
Jump = [] # 只有第一次要跑

In [19]:
cinema = {
    "cinemaId":[],
    "cinemaAddress":[], 
    "cinemaName":[], 
    "cinemaTele":[],
    "cinemaImg":[],
    "cinemaBusTwo":[],
    "cinemaInfo":[],
    "cinemaGoogle":[]  # iframe
}
root = 'https://www.vscinemas.com.tw/vsweb'  # get img url
path = r"C:\Users\User\Desktop\Cinema_Img"
num = 0

# 要新增更多Img部分

In [20]:
for i in tqdm(range (0,100)):
    if i in Jump:
        continue
    resp = rq.get(
        f"https://www.vscinemas.com.tw/vsweb/theater/detail.aspx?id={i}"
    )
    content = BeautifulSoup(resp.text, "html.parser")
    try:
        cinemaTele = content.select_one('div.theaterPosition article.article div.leftBox ul li.icon-phone p').text
        cinemaAddress = content.select_one('div.theaterPosition article.article div.leftBox ul li.icon-marker p').text
        # 交通資訊大家格式都不太一樣，有的有全形的-，有的不是 所以換兩次
        cinemaBusTwo = content.select_one('div.theaterPosition article.article div.leftBox ul li.icon-bus-two p').text.replace(" －",'\n').replace('－',"").replace("-","")
        cinemaGoogle = content.select_one('div.theaterPosition article.article section.rightMap iframe').get('src')
        # 相關資訊也是，每座的影廳都有放不同位置
        cinemaInfo = content.select('div.theaterStory article.article div.bbsArticle p')[0].text
        if cinemaInfo =='\n'or cinemaInfo =='\xa0' or cinemaInfo =='':
            cinemaInfo = content.select('div.theaterStory article.article div.bbsArticle p')[1].text
            
        cinemaName = content.select('section.crumbArea div.crumb span')[2].text
        cinemaName = re.sub('[a-zA-Z]','',cinemaName).replace(" ","")
        Img = content.select_one('section.theaterBanner img').get('src').replace("..",root)
        Img = rq.get(Img)
        if not os.path.exists(path):
            os.mkdir(path)
        with open(f"{path}\_{cinemaName}.jpg","wb") as file:
            file.write(Img.content)
            Img = Image.open(f"{path}\_{cinemaName}.jpg")
            cinemaImg = f"{path}\_{cinemaName}.jpg"
        cinema['cinemaAddress'].append(cinemaAddress)
        cinema['cinemaBusTwo'].append(cinemaBusTwo)
        cinema['cinemaGoogle'].append(cinemaGoogle)
        cinema['cinemaId'].append(i)
        cinema['cinemaImg'].append(cinemaImg)
        cinema['cinemaInfo'].append(cinemaInfo)
        cinema['cinemaName'].append(cinemaName)
        cinema['cinemaTele'].append(cinemaTele)
        num += 1
        if num == 17:
            break
    except:
        Jump.append(i)
        print("Not found：",i)
print("Ending")

 26%|█████████████████████                                                            | 26/100 [00:45<02:08,  1.74s/it]

Ending


In [62]:
Cinema_CSV = pd.DataFrame(cinema)
Cinema_CSV.to_csv("cinema.csv",index=False,encoding="utf_8_sig")

In [22]:
cinema_jason = {'cinemaId':cinema['cinemaId'],'cinemaName':cinema['cinemaName']} # for other jupyter load
with open("cinema.json","wt") as f:
    json.dump(cinema_jason,f)    

In [21]:
cinema['cinemaName']

['台北信義威秀影城',
 '台北京站威秀影城',
 '板橋大遠百威秀影城',
 '林口威秀影城',
 '新竹大遠百威秀影城',
 '新竹巨城威秀影城',
 '頭份尚順威秀影城',
 '台中大遠百威秀影城',
 '台中威秀影城',
 '台南大遠百威秀影城',
 '台南南紡威秀影城',
 '高雄大遠百威秀影城',
 '桃園統領威秀影城',
 '花蓮新天堂樂園威秀影城',
 '台南威秀影城',
 '中和環球威秀影城',
 '台中大魯閣新時代威秀影城']

In [15]:
s = '林口  威秀影城'
s = s.replace(" ","")

In [16]:
s

'林口威秀影城'